# Auto-diff with JAX

JAX is a Python library for accelerator-oriented array computation and program transformation, designed for high-performance numerical computing and large-scale machine learning.

See:

- https://github.com/google/jax
- https://jax.readthedocs.io/en/latest/


In [ ]:
# JAX's syntax is (for the most part) same as NumPy's!
# There is also a SciPy API support (jax.scipy)
import jax
import jax.numpy as jnp
import numpy as np

# JAX's low level API
# (lax is just an anagram for XLA, not completely sure how they came up with name JAX)
from jax import lax

import matplotlib.pyplot as plt

JAX's augmented numpy lives at `jax.numpy`. With a few exceptions, you can think of `jax.numpy` as directly interchangeable with `numpy`. As a general rule, you should use `jax.numpy` whenever you plan to use any of JAX's transformations.


In [ ]:
# Fact 1: JAX's syntax is remarkably similar to NumPy's
x_jnp = jnp.linspace(0, 10, 1000)
y_jnp = 2 * jnp.sin(x_jnp) * jnp.cos(x_jnp)
plt.plot(x_jnp, y_jnp)

In [ ]:
# Fact 2: JAX arrays are immutable!
size = 10
index = 0
value = 23

# In NumPy arrays are mutable
x = np.arange(size)
print(x)
x[index] = value
print(x)

In [ ]:
# In JAX we have to deal with immutable arrays
x = jnp.arange(size)
print(x)
x[index] = value  # ERROR!

In [ ]:
# If this seems wasteful to you, it is indeed true (in normal settings).
# The thing is, if the input value x of x.at[idx].set(y) is not reused,
# you can tell JAX to optimize the array update to occur in-place.
# We will see the details after

jax_array = jnp.zeros((3, 3), dtype=jnp.float32)
updated_array = jax_array.at[1, :].set(1.0)

print("original array unchanged:\n", jax_array)
print("updated array:\n", updated_array)

# The expresiveness of NumPy is still there!

print("new array post-addition:")
new_jax_array = jax_array.at[::2, 1:].add(7.0)
print(new_jax_array)

In [ ]:
# Fact 3: JAX handles random numbers differently (for the same reason arrays are immutable)
seed = 0
key = jax.random.PRNGKey(seed)

x = jax.random.normal(key, (10,))  # you need to explicitly pass the key i.e. PRNG state
print(type(x), x)  # notice the DeviceArray type - that leads us to the next cell!

In [ ]:
# Fact 4: JAX is AI accelerator agnostic. Same code runs everywhere!

size = 3000

# Data is automatically pushed to the AI accelerator (GPU, TPU)
x_jnp = jax.random.normal(key, (size, size), dtype=jnp.float32)
x_np = np.random.normal(size=(size, size)).astype(np.float32)  # some diff in API exists!

print("[1] GPU")
%timeit jnp.dot(x_jnp, x_jnp.T).block_until_ready()  # 1) on GPU - fast
print("[2] Pure numpy (CPU)")
%timeit np.dot(x_np, x_np.T)  # 2) on CPU - slow (NumPy only works with CPUs)
print("[3] GPU + data transfer")
%timeit jnp.dot(x_np, x_np.T).block_until_ready()  # 3) on GPU with transfer overhead

x_np_device = jax.device_put(x_np)  # push NumPy explicitly to GPU
print("[4] GPU + explicit pre-data transfer (like [1] but explicit)")
%timeit jnp.dot(x_np_device, x_np_device.T).block_until_ready()  # same as 1)

# Note1: I'm using GPU as a synonym for AI accelerator. 
# In reality, especially in Colab, this can also be a TPU, etc.

# Note2: block_until_ready() -> asynchronous dispatch

## JAX transform functions

`jit` compiles your functions using XLA and caches them to reach the best speed


In [ ]:
def visualize_fn(fn, l=-10, r=10, n=1000):
  x = np.linspace(l, r, num=n)
  y = fn(x)
  plt.plot(x, y); plt.show()

# Define a function
def selu(x, alpha=1.67, lmbda=1.05):  # note: SELU is an activation function
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

selu_jit = jax.jit(selu)  # let's jit it

# Visualize SELU (just for your understanding, it's always a good idea to visualize stuff)
visualize_fn(selu)

# Benchmark non-jit vs jit version
data = jax.random.normal(key, (1000000,))

print('non-jit version:')
%timeit selu(data).block_until_ready()
print('jit version:')
%timeit selu_jit(data).block_until_ready()

## Automatic differentiation in JAX

JAX augments numpy and Python code with function transformations which make it trivial to perform operations common in machine learning programs.

The function `df = jax.grad(f, argnums = 0)` takes the callable object `f` and returns another callable object, `df`, evaluating the gradient of `f` w.r.t. the argument(s) of index(es) `argnums`. For more information, check out the [documentation](https://jax.readthedocs.io/en/latest/jax.html?highlight=grad#jax.grad).


In [ ]:
x = 1.0  # example input

f = lambda x: x**2 + x + 4  # simple 2nd order polynomial fn
visualize_fn(f, l=-1, r=2, n=100)

dfdx = jax.grad(f)  # 2*x + 1
d2fdx = jax.grad(dfdx)  # 2
d3fdx = jax.grad(d2fdx)  # 0

print(f(x), dfdx(x), d2fdx(x), d3fdx(x))

### What if we had two inputs?

We use `jacrev` and `jacfwd`:

These two functions compute the same values (up to machine numerics), but differ in their implementation: `jacfwd` uses forward-mode automatic differentiation, which is more efficient for “tall” Jacobian matrices, while `jacrev` uses reverse-mode, which is more efficient for “wide” Jacobian matrices. For matrices that are near-square, `jacfwd` probably has an edge over `jacrev`.

To implement hessian, we could have used `jacfwd(jacrev(f))` or `jacrev(jacfwd(f))` or any other composition of the two. But forward-over-reverse is typically the most efficient. That’s because in the inner Jacobian computation we’re often differentiating a function wide Jacobian (maybe like a loss function $f: \mathbb{R}^n \to \mathbb{R}$), while in the outer Jacobian computation we’re differentiating a function with a square Jacobian (since $\nabla f: \mathbb{R}^n \to \mathbb{R}^n$), which is where forward-mode wins out.


In [ ]:
f = lambda x, y: x**2 + y**2  # simple paraboloid

# df/dx = 2x
# df/dy = 2y
# J = [df/dx, df/dy]

# d2f/dx = 2
# d2f/dy = 2
# d2f/dxdy = 0
# d2f/dydx = 0
# H = [[d2f/dx, d2f/dxdy], [d2f/dydx, d2f/dy]]


def eval_hessian(f):
    return jax.jit(jax.jacfwd(jax.jacrev(f, argnums=(0, 1)), argnums=(0, 1)))


jacobian = jax.jacrev(f, argnums=(0, 1))(1.0, 1.0)
hessian = eval_hessian(f)(1.0, 1.0)
print(f"Jacobian = {np.asarray(jacobian)}")
print(f"Hessian = {np.asarray(hessian)}")

In [ ]:
# Naturally, the function passed to grad can work with array inputs
f = lambda x: x[0] * x[0] + x[1] * x[1]


def eval_hessian(f):
    return jax.jit(jax.jacfwd(jax.jacrev(f)))


x0 = jnp.array([2.0, 1.0])
jacobian = jax.jacrev(f)(x0)
hessian = eval_hessian(f)(x0)
print(f"Jacobian = {np.asarray(jacobian)}")
print(f"Hessian = {np.asarray(hessian)}")

In [ ]:
# Edge case |x|, how does JAX handle it?

f = lambda x: abs(x)
visualize_fn(f)

dfdx = jax.grad(f)
print(f"dfdx(0.0)   = {dfdx(0.0)  :.17e}")
print(f"dfdx(+1e-5) = {dfdx(+1e-5):.17e}")
print(f"dfdx(-1e-5) = {dfdx(-1e-5):.17e}")

## Vectorize functions with `vmap()`


In [ ]:
def custom_dot(x, y):
    return jnp.dot(x, y) ** 2

def naive_custom_dot(x_batched, y_batched):
    return jnp.stack([
        custom_dot(v1, v2)
        for v1, v2 in zip(x_batched, y_batched)
    ])

@jax.jit
def jit_naive_custom_dot(x_batched, y_batched):
    return jnp.stack([
        custom_dot(v1, v2)
        for v1, v2 in zip(x_batched, y_batched)
    ])

batched_custom_dot = jax.vmap(custom_dot, in_axes=[0, 0])
jit_batched_custom_dot = jax.jit(jax.vmap(custom_dot, in_axes=[0, 0]))

x = jnp.asarray(np.random.rand(1000, 50))
y = jnp.asarray(np.random.rand(1000, 50))

print("Naive")
%timeit naive_custom_dot(x, y)
print("Vectorized")
%timeit batched_custom_dot(x, y)
print("JIT")
%timeit jit_naive_custom_dot(x, y)
print("Vectorized + JIT")
%timeit jit_batched_custom_dot(x, y)


## The nitty-gritty: JAX API structure


In [ ]:
# Example 1: lax is stricter

print(jnp.add(1, 1.0))  # jax.numpy API implicitly promotes mixed types
print(lax.add(1.0, 1.0))  # works
print(lax.add(1, 1.0))  # ERROR! jax.lax API requires explicit type promotion

In [ ]:
# Example 2: lax is more powerful (but as a tradeoff less user-friendly)

x = jnp.array([1, 2, 1])
y = jnp.ones(10)

# NumPy API
result1 = jnp.convolve(x, y)

# lax API
result2 = lax.conv_general_dilated(
    x.reshape(1, 1, 3).astype(float),  # note: explicit promotion
    y.reshape(1, 1, 10),
    window_strides=(1,),
    padding=[(len(y) - 1, len(y) - 1)],
)  # equivalent of padding='full' in NumPy

print(result1)
print(result2[0][0])
assert np.allclose(result1, result2[0][0], atol=1e-6)

### Gotcha #1: The limitations of JIT

We have used boolean mask extensively, what happens if we use them in JAX?

In [ ]:
def get_negatives(x):
    return x[x < 0]


x = jax.random.normal(key, (10,), dtype=jnp.float32)
print(get_negatives(x))
print(jax.jit(get_negatives)(x))  # ERROR!

The issue comes from the fact that `x[x < 0]` returns an array whose size depends on the values within x; another way of saying this is that `x[x < 0]` has a dynamic shape. JAX's transform model does not currently support dynamically-shaped arrays, and the result is the error you see.


In [ ]:
# So how does it work in the background? -> tracing on different levels of abstraction


@jax.jit
def f(x, y):
    print("Running f():")
    print(f"  x = {x}")
    print(f"  y = {y}")
    result = jnp.dot(x + 1, y + 1)
    print(f"  result = {result}")
    return result


x = np.random.randn(3, 4)
y = np.random.randn(4)
print(f(x, y))  # The first call has the overhead of compiling the function


x2 = np.random.randn(3, 4)
y2 = np.random.randn(4)
print("Second call:")
print(f(x2, y2))  # Oops! Side effects (like print) are not compiled...

In [ ]:
# Same function as above just without the print statements
def f(x, y):
    return jnp.dot(x + 1, y + 1)


print(jax.make_jaxpr(f)(x, y))

In [ ]:
# 2nd example of a failure:


@jax.jit
def f(x, neg):  # result depends on the value - remember tracer cares about shapes and types!
    return -x if neg else x

@jax.jit
def f2(x, neg):
    return x * (1.0 - 2.0 * neg)

f2(x, False)
f(1, True) # ERROR!

In [ ]:
# Workaround: the "static" arguments

from functools import partial


@partial(jax.jit, static_argnums=(1,))
def f(x, neg):
    print(x)
    return -x if neg else x


print(f(1, True))
print(f(2, True))
# Here we do another jit compilation
print(f(2, False))
print(f(23, False))
# Here we do another jit compilation
print(f(44, 1))

JAX is designed to work only on pure functions!

Pure function? Informal definition:

- All the input data is passed through the function parameters, all the results are output through the function results.
- A pure function will always return the same result if invoked with the same inputs.


In [ ]:
# Example 3

g = 0.0


def impure_uses_globals(x):
    return x + g  # Violating both #1 and #2


# JAX captures the value of the global during the first run
print("First call: ", jax.jit(impure_uses_globals)(4.0))

# Let's update the global!
g = 10.0

# Subsequent runs may silently use the cached value of the globals
print("Second call: ", jax.jit(impure_uses_globals)(5.0))

# JAX re-runs the Python function when the type or shape of the argument changes
# This will end up reading the latest value of the global
print("Third call, different type: ", jax.jit(impure_uses_globals)(jnp.array([4.0])))

### Gotcha #2: Out-of-Bounds Indexing

Due to JAX's accelerator agnostic approach JAX had to make a non-error behaviour for out of bounds indexing (similarly to how invalid fp arithmetic results in NaNs and not an exception).


In [ ]:
# JAX behavior
# 1) updates at out-of-bounds indices are skipped
# 2) retrievals result in index being clamped
# in general there are currently some bugs so just consider the behavior undefined!

print(jnp.arange(10).at[11].add(23))  # example of 1)
print(jnp.arange(10)[11])  # example of 2)

### Gotcha #3: random numbers


In [ ]:
# NumPy - PRNG is stateful!
seed = 0

# Let's sample calling the same function twice
np.random.seed(seed)

rng_state = np.random.get_state()
print("numpy random state", rng_state[2:])

print("A random number", np.random.random())
rng_state = np.random.get_state()
print("numpy random state",rng_state[2:])

print("A random number", np.random.random())
rng_state = np.random.get_state()
print("numpy random state",rng_state[2:])

In [ ]:
# JAX's random functions can't modify PRNG's state!

key = jax.random.PRNGKey(seed)
print("JAX rng state", key)  # key defines the state (2 unsigned int32s)

# Let's again sample calling the same function twice
print("A random number", jax.random.normal(key, shape=(1,)))
print("JAX rng state",key)  # verify that the state hasn't changed

print("A random number", jax.random.normal(key, shape=(1,)))  # oops - same results?
print("JAX rng state",key)

In [ ]:
# Solution? -> Split every time you need a pseudorandom number.

print("old key", key)
key, subkey = jax.random.split(key, 2)
normal_pseudorandom = jax.random.normal(subkey, shape=(1,))
print("    \---SPLIT --> new key   ", key)
print("             \--> new subkey", subkey, "--> normal", normal_pseudorandom)

Why this design?
With current design can the code be easily reproducible in a parallel environment?

E.g. imagine two functions that run on two different CPU/GPU (in a parallel env, where you are separately training a neural network), than the result may depend on who first calls the np.random.uniform between the two CPU/GPU

In [ ]:
import threading
import time

# simulate the training on CPU/GPU #1
def worker1(result_container):
    for _ in range(10):
        result_container["worker1"].append(np.random.uniform())
        time.sleep(0.001)

# simulate the training on CPU/GPU #2
def worker2(result_container):
    for _ in range(10):
        result_container["worker2"].append(np.random.uniform())
        time.sleep(0.002)


def do_parallel_work():
    result = {"worker1": [], "worker2": []}
    t1 = threading.Thread(target=worker1, args=(result,))
    t2 = threading.Thread(target=worker2, args=(result,))

    # Start both threads "at the same time"
    t1.start()
    t2.start()

    # Wait for both to finish
    t1.join()
    t2.join()

    # combine the work
    return (np.array(result["worker1"]) + 2 * np.array(result["worker2"])).sum()


# What if we want to parallelize this code? NumPy assumes too much.
# Run do_parallel_work() multiple times to show non-determinism
for i in range(5):
    np.random.seed(42)
    print(f"Run {i+1}: {do_parallel_work()}")



### Gotcha #4: Control Flow


In [ ]:
# Python control flow + grad() -> everything is ok
def f(x):
    if x < 3:
        return 3.0 * x**2
    else:
        return -4 * x


x = np.linspace(-10, 10, 1000)
y = [f(el) for el in x]

print(jax.grad(f)(2.0))  # ok!
print(jax.grad(f)(4.0))  # ok!

In [ ]:
# Python control flow + jit() -> problems in paradise.

# "The tradeoff is that with higher levels of abstraction we gain a more general view
# of the Python code (and thus save on re-compilations),
# but we require more constraints on the Python code to complete the trace."

# Solution (recall: we already have seen this)
f_jit = jax.jit(f, static_argnums=(0,))
x = 2.0

print(jax.make_jaxpr(f_jit, static_argnums=(0,))(x))
print(f_jit(x))

# WARNING: still for each x we have to jit a new function, this might be expensive!

f_jit_error = jax.jit(f)
f_jit_error(2.0)

In [ ]:
# native jax functions give you powerfull alternatives

def f(x):
    return jnp.where(x < 3.0, 3.0 * x**2, -4 * x)

f_jit = jax.jit(f)
fgrad_jit = jax.jit(jax.grad(f))

print(f_jit(2.0))
print(f_jit(4.0))
print(fgrad_jit(2.0))
print(fgrad_jit(4.0))


In [ ]:
# Example 2: range depends on value again


def f(x, n):
    y = 0.0
    for i in range(n):
        y = y + x[i]
    return y


f_jit = jax.jit(f, static_argnums=(1,))
x = (jnp.array([2.0, 3.0, 4.0]), 3)

print(jax.make_jaxpr(f_jit, static_argnums=(1,))(*x))  # notice how for loop gets unrolled
print(f_jit(*x))

# Note: there is a catch - static args should not change a lot!

In [ ]:
# Even "better" solution is to use low level API
def f_fori(x, n):
    body_fun = lambda i, val: val + x[i]
    return lax.fori_loop(0, n, body_fun, 0.0)


f_fori_jit = jax.jit(f_fori)

print(jax.make_jaxpr(f_fori_jit)(*x))
print(f_fori_jit(*x))

### Other gotchas


In [ ]:
# If you want to debug where the NaNs are coming from, there are multiple ways
# to do that, here is one:
jax.config.update("jax_debug_nans", False)  # Change this flag and re-run the cell
jnp.divide(0.0, 0.0)

In [ ]:
# JAX enforces single precision! There are simple ways around it though.
jax.config.update("jax_enable_x64", True)
x = jax.random.uniform(key, (1000,), dtype=jnp.float64)
print(x.dtype)